## Architektury agentów

### Instalacja bibliotek

In [ ]:
%pip install -U langgraph langchain langchain-openai

### Import bibliotek i konfiguracja

In [1]:
import operator
from typing import Annotated, List, TypedDict, Literal

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


### Sequential Agent
Sequential Agent prowadzi wewnętrzne rozumowanie w kilku krokach (tzw. scratchpad), ale nie używa narzędzi; stosowany tam, gdzie wystarczy czysta analiza i dedukcja.

In [ ]:

class CoTState(TypedDict):
    question: str
    # Accumulate steps across nodes if desired
    steps: Annotated[List[str], operator.add]
    answer: str

def plan_node(state: CoTState) -> dict:
    """Ask the LLM to propose a compact plan (steps) without solving yet."""
    sys = (
        "You are a careful planner. Break the user's question into 2-4 concise steps. "
        "Do not solve. Return only a numbered list of steps; no extra text."
    )
    messages = [("system", sys), ("user", state["question"])]
    resp = llm.invoke(messages)
    # Normalize into a list of strings (one per step)
    raw = resp.content
    steps = []
    for line in str(raw).splitlines():
        line = line.strip()
        if not line:
            continue
        # strip possible numbering like "1. ", "- ", etc.
        line = line.lstrip("-• ").split(". ", 1)[-1] if ". " in line[:4] else line.lstrip("-• ")
        steps.append(line)
    return {"steps": steps}

def solve_node(state: CoTState) -> dict:
    """Use the planned steps to derive the final answer only."""
    sys = (
        "Use the provided steps to solve the problem. "
        "Return only the final answer, no reasoning."
    )
    messages = [
        ("system", sys),
        ("user", f"Question: {state['question']}\nSteps: {state['steps']}"),
    ]
    resp = llm.invoke(messages)
    return {"answer": str(resp.content).strip()}

# Wire up the graph
graph = StateGraph(CoTState)
graph.add_node("plan", plan_node)
graph.add_node("solve", solve_node)
graph.add_edge(START, "plan")
graph.add_edge("plan", "solve")
graph.add_edge("solve", END)
cot_graph = graph.compile()


In [ ]:
# Run the CoT graph
state = {
    "question": "If a book has 350 pages and I read 14 pages per day, how many days to finish?",
    "steps": [],
    "answer": ""
}
out = cot_graph.invoke(state)
print("Final answer:", out["answer"])

# If you're teaching and want to peek at the internal scratchpad (not for end users):
# print("Planned steps:", out["steps"])


### Custom Agent
Custom Agent daje pełną elstyczność, można samemu zdefiniować logikę, routing i węzły.

In [ ]:

class CustomState(TypedDict):
    input: str
    task: Literal["math", "capitalize", "count"]
    result: str

def route(state: CustomState) -> str:
    """Deterministic router based on a simple protocol in the input."""
    text = state["input"].strip().lower()
    if text.startswith("math:"):
        return "math"
    if text.startswith("capitalize:"):
        return "capitalize"
    if text.startswith("count:"):
        return "count"
    # default fallback
    return "count"

def do_math(state: CustomState) -> dict:
    expr = state["input"].split(":", 1)[-1].strip()
    allowed = set("0123456789+-*/(). ")
    if any(c not in allowed for c in expr):
        return {"result": "Error: unsupported characters in math expression."}
    try:
        res = eval(expr, {"__builtins__": {}})
    except Exception as e:
        res = f"Error: {e}"
    return {"result": str(res)}

def do_capitalize(state: CustomState) -> dict:
    text = state["input"].split(":", 1)[-1].strip()
    return {"result": text.upper()}

def do_count(state: CustomState) -> dict:
    text = state["input"].split(":", 1)[-1].strip()
    tokens = [t for t in text.split() if t]
    return {"result": f"words={len(tokens)} chars={len(text)}"}

g = StateGraph(CustomState)
g.add_node("math", do_math)
g.add_node("capitalize", do_capitalize)
g.add_node("count", do_count)

# Conditional edges from router
g.add_conditional_edges(
    START,
    route,
    {
        "math": "math",
        "capitalize": "capitalize",
        "count": "count",
    },
)
g.add_edge("math", END)
g.add_edge("capitalize", END)
g.add_edge("count", END)

custom_agent = g.compile()


In [ ]:

# Try several inputs
for user_input in [
    "math: (12 + 8) * 3",
    "capitalize: langgraph is great!",
    "count: How many words are here?",
]:
    out = custom_agent.invoke({"input": user_input, "task": "count", "result": ""})
    print(f"Input: {user_input}\nResult: {out['result']}\n---")


### Supervisor

### Router